# Demo: Run bedrock model and ingest input and output token count into Amberflo

## 1. Install dependencies

The following is a list of requirements needed for reproducing this notebook on a local environment.
```
amberflo-metering-python==3.3.1
boto3==1.37.11
```

You can put the following on a requirements.txt, and then run `pip install -r requirements.txt`.

## 2. Setup your credentials

The demo will use the AWS services:

- bedrock: for answering a prompt
- s3: in order to ingest data data into Amberflo

If you leave the following variables set to `None`, then the AWS clients will try to automatically pick up your credentials.

Depending on your runtime, you might need to set up these explicitly.

In [ ]:
REGION = "us-west-2"
aws_access_key_id=None # or "<YOUR AWS KEY ID>"
aws_secret_access_key=None # or "<YOUR AWS SECRET KEY>"
amberflo_ingestion_bucket="<YOUR INGESTION BUCKET>" # get your bucket name from customer support

## 3. Import relevant libraries

In [ ]:
import boto3
import json
import logging
from time import time
from metering.ingest import create_ingest_client, create_ingest_payload

## 4. Initialize long living resources

The code below will create clients for Amazon Bedrock and for Amberflo.

Note that the Amberflo client is configured to push data to S3 on a background thread. This is the optimal configuration for high trhouput ingestion. For alternative initializations see  https://docs.amberflo.io/docs/python-library.

In [ ]:
bedrock_client = boto3.client(
    "bedrock-runtime",
    region_name=REGION,
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
)

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

metering_client = create_ingest_client(
    bucket_name=amberflo_ingestion_bucket,
    access_key=aws_access_key_id,
    secret_key=aws_secret_access_key,
)

## 5. Run your LLM model

In [ ]:
def get_chat_completion(prompt):
    return bedrock_client.invoke_model(
        modelId="anthropic.claude-3-5-haiku-20241022-v1:0",
        body=json.dumps(
            {
                "anthropic_version": "bedrock-2023-05-31",
                "messages": [{"role": "user", "content": prompt}],
                "max_tokens": 256,
                "temperature": 0.7,
            }
        ),
    )

# LLM input
prompt = "What are the key difficulties in managing my AI budget?"

# Invoke the AWS Bedrock model
response = get_chat_completion(prompt)
llm_result = json.loads(response["body"].read().decode("utf-8"))
print(json.dumps(llm_result, indent=2))

## 6. Ingest your meter

In [ ]:
def ingest_llm_response(llm_result, metering_client, llm_context, caller_context):
    input_tokens = llm_result.get("usage", {}).get("input_tokens", 0)
    output_tokens = llm_result.get("usage", {}).get("output_tokens", 0)
    content_items = llm_result.get("content", [])
    content_type = (
        content_items[0].get("type", "unknown") if content_items else "unknown"
    )

    # Construct meter event dimensions
    dimensions = {
        "model": llm_result.get("model"),
        "provider": llm_context.get("provider"),
        "region": llm_context.get("region"),
        "batch": llm_context.get("batch"),
        "model_type": content_type,
        "user_id": caller_context.get("user_id"),
        "app": caller_context.get("app"),
    }

    logger.info(
        "AFLO_LLM: %s, afloDimensions: %s",
        json.dumps(llm_result, indent=4),
        json.dumps(dimensions, indent=4),
    )

    for meter_api_name, meter_value in [
        ("amazon_bedrock_input_tokens", input_tokens),
        ("amazon_bedrock_output_tokens", output_tokens),
    ]:
        event = create_ingest_payload(
            meter_api_name=meter_api_name,
            meter_time_in_millis=int(round(time() * 1000)),
            meter_value=meter_value,
            customer_id="sales",
            dimensions=dimensions,
            # unique_id is the LLM response id
            # and can link input and output tokens
            unique_id=llm_result.get("id"),
        )
        metering_client.send(event)

caller_context = {
    "user_id": "talha",
    "app": "sales-chatbot",
    "department": "sales",
}

llm_context = {
    "batch": "false",
    "provider": "antrhopic",
    "region": REGION
}

ingest_llm_response(llm_result, metering_client, llm_context, caller_context)

## 6. Shutdown the Amberflo client

This operation will make sure that any pending meter event was properly submit to Amberflo.

In [ ]:
metering_client.shutdown()